In [1]:
import gc

import numpy as np

"""
ToyNetwork 재분석
    램프 : 진입
    집계시간 : 5분
    분석시간 1800~8400
    검지기 : 841개
"""
import pandas as pd

import os

# FIX 값 모음
###################################################################################################################

path = r"C:\VISSIM_Workspace\테스트\램프0_진입램프\mer 통합"

start_interval = 1800
end_interval = 8400

weights = {
    "w1" : 1,
    "w2" : 1,
    "w3" : 1,
    "w4" : 1,
    "w5" : 1,
    "w6" : 1
}

vehicle_types = [100, 300, 630, 640, 650]

enter_line = [1]

# 램프 전 본선 검지기
before_ramp = [49]

# 램프 후 본선 검지기
after_ramp = [53]

# 유입 검지기
input_ramp = [999]

# 유출 검지기
output_ramp = [0]

###################################################################################################################

# 함수 모음
###################################################################################################################

# 속도 변화율
def speed_mean(original_df):
    copy_df = original_df.copy()

    # 램프 검지기 제외
    copy_df = copy_df[~copy_df["New_Measurement"].between(900, 910)]

    # TimeGroup, New_Measurement별 그룹화 및 속도 평균
    speed_mean_df = (
        copy_df.groupby(["TimeGroup", "New_Measurement"])
          .agg(V_mean=("v[km/h]", "mean"), V_count=("v[km/h]", "count"))
          .reset_index()
    )
    speed_mean_df["V_next"] = speed_mean_df.groupby("TimeGroup")["V_mean"].shift(-1)
    speed_mean_df["delta_V"] = (speed_mean_df["V_next"] - speed_mean_df["V_mean"]) / speed_mean_df["V_mean"]
    speed_mean_df["delta_V"] = speed_mean_df["delta_V"].fillna(0)

    return speed_mean_df

# 밀도 변화율
def density_mean(speed_df):
    copy_df = speed_df.copy()
    density_mean_df = copy_df.assign(K = copy_df["V_count"] * 12 / copy_df["V_mean"])
    density_mean_df["K_next"] = density_mean_df.groupby("TimeGroup")["K"].shift(-1)
    density_mean_df["delta_K"] = (density_mean_df["K_next"] - density_mean_df["K"]) / density_mean_df["K"]
    density_mean_df["delta_K"] = density_mean_df["delta_K"].fillna(0)
    return density_mean_df

# 중차량 혼입률
def heavy_rate(original_df):
    copy_df = original_df.copy()

    heavy_df = (
        copy_df[copy_df["Vehicle type"].isin([630, 640, 650])]
        .groupby(["TimeGroup"])
        .size()
        .reset_index(name="heavy_count")
    )

    # TimeGroup별 총 차량 갯수 집계
    total_df = (
        copy_df.groupby(["TimeGroup"])
        .size()
        .reset_index(name="total_count")
    )

    heavy_rate_df = pd.merge(
        heavy_df,
        total_df,
        on=["TimeGroup"],
        how="left"
    )

    heavy_rate_df["rate"] = heavy_rate_df["heavy_count"] / heavy_rate_df["total_count"]
    return heavy_rate_df


# 동적 포화도
def entry_saturation(original_df):
    copy_df = original_df.copy()

    # 실측용량 C(3차로 6600)
    max_capacity = 6600
    entry_saturation_df = (
        copy_df.groupby(["TimeGroup", "New_Measurement"])
        .size()
        .reset_index(name="entry_volume")  # 차량 수를 entry_volume이라는 컬럼명으로
    )

    # 단위가 대/시 이기 때문에 현재 5분집계 * 12
    entry_saturation_df["Phi_진입"] = entry_saturation_df["entry_volume"] * 12 / max_capacity
    return entry_saturation_df

# 램프 간섭 영향률
def rfr_rate(original_df):
    copy_df = original_df.copy()
    copy_df["TimeGroup"] = copy_df["TimeGroup"].astype(str)
    main_results=[]
    for i, (before, after) in enumerate(zip(before_ramp, after_ramp)):
        q_before = (copy_df[copy_df["New_Measurement"] == before]
                    .groupby("TimeGroup")
                    .size()
                    .reset_index(name="q_before"))

        q_after = (copy_df[copy_df["New_Measurement"] == after]
                   .groupby("TimeGroup")
                   .size()
                   .reset_index(name="q_after"))

        merged = q_before.merge(q_after, on="TimeGroup", how="outer").fillna(0)
        merged["Qm"] = (merged["q_before"] + merged["q_after"]) / 2
        main_results.append(merged)

    main_df = pd.concat(main_results, ignore_index=True)
    main_df = main_df.groupby("TimeGroup")[["Qm"]].mean().reset_index()

    ramp_results = []
    for input_, output_ in zip(input_ramp, output_ramp):
        q_in = (copy_df[copy_df["New_Measurement"] == input_]
                .groupby("TimeGroup").size().reset_index(name="q_in"))

        q_out = (copy_df[copy_df["New_Measurement"] == output_]
                 .groupby("TimeGroup").size().reset_index(name="q_out"))
        q_out["q_out"] = 0
        merged = q_in.merge(q_out, on="TimeGroup", how="outer").fillna(0)
        ramp_results.append(merged)

    ramp_df = pd.concat(ramp_results, ignore_index=True)
    ramp_df = ramp_df.groupby("TimeGroup")[["q_in", "q_out"]].mean().reset_index()

    rfr_df = pd.merge(main_df, ramp_df, on="TimeGroup", how="inner")

    # 램프 간섭 영향률 계산
    rfr_df["IR_in"] = rfr_df["q_in"] / rfr_df["Qm"]
    rfr_df["IR_out"] = rfr_df["q_out"] / rfr_df["Qm"]
    rfr_df["RFR"] = rfr_df["IR_in"] + rfr_df["IR_out"]

    # -----------------------------
    # 특정 검지기에만 RFR 반영
    # -----------------------------
    target_measurements = after_ramp   # RFR 적용 대상 검지기 번호
    all_measurements = copy_df["New_Measurement"].unique()

    expanded_df_list = []
    for m in all_measurements:
        temp = rfr_df.copy()
        temp["New_Measurement"] = m
        # 지정된 구간에만 RFR 값 유지, 나머지는 0
        temp["RFR"] = temp["RFR"] if m in target_measurements else 0
        expanded_df_list.append(temp)

    final_rfr_df = pd.concat(expanded_df_list, ignore_index=True)
    final_rfr_df = final_rfr_df.sort_values(by=["TimeGroup", "New_Measurement"]).reset_index(drop=True)
    #final_rfr_df["RFR"] = 0
    return final_rfr_df

# 진출 원활율
def output_normality(original_df):
    copy_df = original_df.copy()
    entry_df = copy_df[copy_df["New_Measurement"].isin(enter_line)][["VehNo", "t(Entry)"]]

    exit_df = copy_df[copy_df["New_Measurement"].isin(before_ramp)][["VehNo", "t(Entry)"]]

    # 차량 번호로 그룹화 후 시간의 최솟값(중복제거)
    entry_first = (
        entry_df.groupby("VehNo")["t(Entry)"].min()
        .reset_index()  # Series → DataFrame
        .rename(columns={"t(Entry)": "t_entry"})
    )

    exit_first = (
        exit_df.groupby("VehNo")["t(Entry)"].min()
        .reset_index()
        .rename(columns={"t(Entry)": "t_exit"})
    )

    # 지연시간
    merged = pd.merge(entry_first, exit_first, on="VehNo", how="inner")
    merged["delay_sec"] = merged["t_exit"] - merged["t_entry"]
    merged = merged[merged["delay_sec"] >= 0]  # 음수 제거

    # 지연시간(중앙값) → lag_bins
    if len(merged) and np.isfinite(np.nanmedian(merged["delay_sec"])): # delay_sec의 값이 유효하면
        lag_bins = int(round(np.nanmedian(merged["delay_sec"]) / 300)) # 단위시간으로 나눴을 때의 중간값 => 3(900초) => 진입한 차량이 진출을 통과하는데 평균 900초가 걸림
    else:
        lag_bins = 0  # 데이터 부족 시 동시간 매칭

    # TimeGroup별 진입/유출 카운트 집계
    entry_count = (original_df[original_df["New_Measurement"].isin(enter_line)]
                .groupby("TimeGroup").size().reset_index(name="Q_in"))
    exit_count  = (original_df[original_df["New_Measurement"].isin(before_ramp)]
                .groupby("TimeGroup").size().reset_index(name="Q_out"))

    merged_counts = pd.merge(entry_count, exit_count, on="TimeGroup", how="left")

    # Q_out을 지연 시간만큼 shift
    merged_counts["Q_out_shift"] = merged_counts["Q_out"].shift(-lag_bins)

    # F(outrate) 계산
    merged_counts["F(outrate)"] = (merged_counts["Q_out_shift"] / merged_counts["Q_in"]).fillna(0)

    all_measurements = copy_df["New_Measurement"].unique()
    expanded_list = []

    for m in all_measurements:
        temp = merged_counts.copy()
        temp["New_Measurement"] = m
        if m not in before_ramp:  # 222 이외의 검지기
            temp["F(outrate)"] = 0
        expanded_list.append(temp)

    final_df = pd.concat(expanded_list, ignore_index=True)
    final_df = final_df.sort_values(by=["TimeGroup", "New_Measurement"]).reset_index(drop=True)
    final_df["F(outrate)"] = 0
    return final_df


def calculate_stvm(speed_df, density_df, heavy_df, entry_saturation_df, rfr_df, normality_df):

    # TimeGroup 기준으로  Merge
    merged_df = (
        speed_df[["TimeGroup", "New_Measurement", "delta_V"]]
        .merge(density_df[["TimeGroup", "New_Measurement", "delta_K"]], on=["TimeGroup", "New_Measurement"])
        .merge(heavy_df[["TimeGroup", "rate"]], on=["TimeGroup"])
        .merge(entry_saturation_df[["TimeGroup", "New_Measurement", "Phi_진입"]], on=["TimeGroup", "New_Measurement"])
        .merge(rfr_df[["TimeGroup", "New_Measurement", "RFR"]], on=["TimeGroup", "New_Measurement"])
        .merge(normality_df[["TimeGroup", "New_Measurement", "F(outrate)"]], on=["TimeGroup", "New_Measurement"])
    )

    merged_df["STVM"] = (
        weights["w1"] * merged_df["delta_V"] +
        weights["w2"] * merged_df["delta_K"] +
        weights["w3"] * merged_df["rate"] +
        weights["w4"] * merged_df["Phi_진입"] +
        weights["w5"] * merged_df["RFR"] +
        weights["w6"] * merged_df["F(outrate)"]
    )
    merged_df = modify_frame(merged_df)
    return merged_df


def calculate_z_score(stvm_df):
    copy_df = stvm_df.copy()

    # 평균
    mean_stvm = copy_df["STVM"].mean(skipna=True)

    # 표준편차
    std_stvm = copy_df["STVM"].std(skipna=True)

    # Z-Score 계산
    copy_df["Z-Score"] = (copy_df["STVM"] - mean_stvm) / std_stvm
    z_max = copy_df["Z-Score"].max()
    z_min = copy_df["Z-Score"].min()

    copy_df["환산점수"] = copy_df["Z-Score"].apply(lambda z : z_to_score(z, z_min, z_max))

    stvm_df = pd.pivot(copy_df, index="TimeGroup", columns= "New_Measurement", values="환산점수")
    return stvm_df

def modify_frame(df):
    modify_df = df.copy()

    modify_df["StartTime"] = modify_df["TimeGroup"].str.split("~").str[0].astype(int)
    modify_df = modify_df[(modify_df["StartTime"] >=1800) &(modify_df["StartTime"] < 5400)]
    modify_df = modify_df[~modify_df["New_Measurement"].isin([280,999])]
    return modify_df


def variable_timegroup_avg(stvm_df):
    copy_df = stvm_df.copy()
    variable_time_df = copy_df.groupby("TimeGroup")[["delta_V", "delta_K", "rate", "Phi_진입", "RFR", "F(outrate)"]].mean()
    return variable_time_df

def variable_total_avg(variable_df):
    variable_total_df = pd.DataFrame([variable_df.mean(numeric_only=True)])
    return variable_total_df

def speed_density_avg(density_df):
    copy_df = density_df.copy()
    avg_df = modify_frame(copy_df)
    avg_df = pd.DataFrame([avg_df.mean(numeric_only=True)])
    avg_df = avg_df[["V_mean", "K"]]
    return avg_df

def pivot_table(df, value, preprocess=None):
    copy_df = df.copy()
    if preprocess :
        copy_df = preprocess(copy_df)
    return copy_df.pivot(index="TimeGroup", columns="New_Measurement", values=value)

def weighted_avg_speed(original_df):
    copy_df = original_df.copy()
    # TimeGroup, New_Measurement별 그룹화 및 속도 평균
    speed_mean_df = (
        copy_df.groupby(["TimeGroup", "New_Measurement", "Vehicle type"])
          .agg(V_mean=("v[km/h]", "mean"), V_count=("v[km/h]", "count"))
          .reset_index()
    )
    speed_mean_df["std_group"] = speed_mean_df.groupby(["TimeGroup", "New_Measurement"])["V_mean"].transform(lambda s: s.std(ddof=0))
    speed_mean_df["cv"] = speed_mean_df["std_group"] / speed_mean_df["V_mean"]
    speed_mean_df["w"] = 1 / speed_mean_df["cv"]
    speed_mean_df["w*v"] = speed_mean_df["w"] * speed_mean_df["V_mean"]

    weighted_result = (
        speed_mean_df.groupby(["TimeGroup","New_Measurement"])
          .apply(lambda g: g["w*v"].sum() / g["w"].sum())
          .reset_index(name="Weighted_Avg_Speed")
    )

    return weighted_result

def save_to_excel(excel_df, folder_path, file_name, i):
        excel_folder_path = os.path.join(folder_path, file_name)
        os.makedirs(excel_folder_path, exist_ok=True)
        excel_file_name = f"{file_name}_{i+1}.xlsx"
        excel_file_path = os.path.join(excel_folder_path, excel_file_name)
        excel_df.to_excel(excel_file_path, index=True)
        print(f"{excel_file_name} 생성 완료")

def z_to_score(z, z_min, z_max):
    if 1.645 <= z <= z_max:
        return 50 + ((95 + 5 * ((z - 1.645) / (z_max - 1.645))) * 0.5)
    elif 1.282 <= z < 1.645:
        return 50 + ((90 + 5 * ((z - 1.282) / (1.645 - 1.282))) * 0.5)
    elif 1.038 <= z < 1.282:
        return 50 + ((85 + 5 * ((z - 1.038) / (1.282 - 1.038))) * 0.5)
    elif 0.842 <= z < 1.038:
        return 50 + ((80 + 5 * ((z - 0.842) / (1.038 - 0.842))) * 0.5)
    elif 0.676 <= z < 0.842:
        return 50 + ((75 + 5 * ((z - 0.676) / (0.842 - 0.676))) * 0.5)
    elif 0.526 <= z < 0.676:
        return 50 + ((70 + 5 * ((z - 0.526) / (0.676 - 0.526))) * 0.5)
    elif 0.387 <= z < 0.526:
        return 50 + ((65 + 5 * ((z - 0.387) / (0.526 - 0.387))) * 0.5)
    elif 0.255 <= z < 0.387:
        return 50 + ((60 + 5 * ((z - 0.255) / (0.387 - 0.255))) * 0.5)
    elif -0.255 <= z < 0.255:
        return 50 + ((40 + 5 * ((z + 0.255) / (0.255 + 0.255))) * 0.5)
    elif -0.387 <= z < -0.255:
        return 50 + ((35 + 5 * ((z + 0.387) / (-0.255 + 0.387))) * 0.5)
    elif -0.526 <= z < -0.387:
        return 50 + ((30 + 5 * ((z + 0.526) / (-0.387 + 0.526))) * 0.5)
    elif -0.676 <= z < -0.526:
        return 50 + ((25 + 5 * ((z + 0.676) / (-0.676 + 0.842))) * 0.5)
    elif -0.842 <= z < -0.676:
        return 50 + ((20 + 5 * ((z + 0.842) / (-0.676 + 0.842))) * 0.5)
    elif -1.038 <= z < -0.842:
        return 50 + ((15 + 5 * ((z + 1.038) / (-0.842 + 1.038))) * 0.5)
    elif -1.282 <= z < -1.038:
        return 50 + ((10 + 5 * ((z + 1.282) / (-1.038 + 1.282))) * 0.5)
    elif -1.645 <= z < -1.282:
        return 50 + ((5 + 5 * ((z + 1.645) / (-1.282 + 1.645))) * 0.5)
    elif z_min <= z < -1.645:
        return 50 + ((0 + 5 * ((z + z_min) / (-1.645 + z_min))) * 0.5)
    else:
        return np.nan
###################################################################################################################

folder_path = path
mer_list = [file for file in os.listdir(folder_path) if file.endswith(".mer")]

grouped_df = pd.DataFrame()
result_df = pd.DataFrame()


for i in range(len(mer_list)):
    mer_file = mer_list[i]
    with open(os.path.join(folder_path, mer_file), "r", encoding="utf-8", errors="ignore") as file:
            lines = file.readlines()
            # 데이터가 시작하는 인덱스 찾기
            data_start_idx = None

            for j, line in enumerate(lines):
                if "Measurem." in line:  # 컬럼명이 포함된 행 찾기
                    data_start_idx = j
                    break

            # 데이터프레임 생성
            if data_start_idx is not None:

                # 컬럼명 추출 및 공백 제거
                columns = [col.strip() for col in lines[data_start_idx].strip().split(";")]

                # 데이터 부분 추출 및 가공
                data_lines = lines[data_start_idx + 1:]  # 컬럼명 제외, 데이터 부분
                data = [line.strip().split(";") for line in data_lines if line.strip()]

                # 데이터프레임 생성
                df = pd.DataFrame(data, columns=columns)

                # 컬럼 내부 데이터 정수형 변환
                df = df.apply(pd.to_numeric, errors="coerce")

                original_df = df[(df["t(Entry)"] != -1.00)].reset_index(drop=True)


                #불필요 컬럼 제거
                original_df.drop(columns=["b[m/s2]", "tQueue", "Occ", "Pers"], inplace=True, errors="ignore")

                original_df["New_Measurement"] = original_df["Measurem."] % 1000

                bins = np.arange(start_interval, end_interval+1, 300)
                labels = [f"{start}~{start+300}" for start in bins[:-1]]  # 구간 라벨링

                # 구간 나누기 및 컬럼 추가
                original_df["TimeGroup"] = pd.cut(original_df["t(Entry)"], bins=bins, labels=labels, right=False)

                # 속도변화율
                speed_df = speed_mean(original_df)

                # 밀도변화율
                density_df = density_mean(speed_df)

                # 중차량 혼입률
                heavy_df = heavy_rate(original_df)

                # 동적 포화도
                entry_saturation_df = entry_saturation(original_df)

                # 램프 간섭 영향률
                rfr_df = rfr_rate(original_df)

                # 진출 원활율
                normality_df = output_normality(original_df)

                # STVM 계산
                stvm_df = calculate_stvm(speed_df, density_df, heavy_df, entry_saturation_df, rfr_df, normality_df)
                #display("stvm_df : ", stvm_df)

                # Z-Score 계산
                z_score_df = calculate_z_score(stvm_df)
                display("z_score_df : ", z_score_df)
                save_to_excel(z_score_df, folder_path, "환산점수", i)

                # STVM 피봇
                #stvm_pivot_df = pivot_table(stvm_df, "STVM")
                #display("stvm_pivot_df : ", stvm_pivot_df)
                #save_to_excel(stvm_pivot_df, folder_path, "STVM", i)

                # 속도값 피봇
                #speed_pivot_df = pivot_table(speed_df, "V_mean", preprocess=modify_frame)
                #display("speed_pivot_df : ", speed_pivot_df)
                #save_to_excel(speed_pivot_df, folder_path, "속도값", i)

                # 메모리 정리
                #del df, original_df, speed_df, density_df, heavy_df, entry_saturation_df, rfr_df, normality_df, stvm_df, z_score_df
                gc.collect()


'z_score_df : '

New_Measurement,1,2,3,4,5,6,7,8,9,10,...,270,271,272,273,274,275,276,277,278,279
TimeGroup,,,,,,,,,,,,,,,,,,,,,
1800~2100,71.084554,71.083082,71.091650,71.081580,71.075975,71.080344,71.080349,71.083198,71.073195,71.063293,...,71.171788,71.155525,71.153278,71.166394,71.162490,71.164805,71.166201,71.160592,71.160173,71.163579
2100~2400,71.066569,71.071183,71.071185,71.068228,71.078781,71.076856,71.078224,71.076659,71.073748,71.089860,...,71.167975,71.185449,71.186993,71.178991,71.176204,71.173438,71.177495,71.183382,71.171701,71.184373
2400~2700,71.060731,71.052387,71.057215,71.046211,71.051379,71.048241,71.046858,71.058106,71.058980,71.043121,...,71.191142,71.191021,71.191000,71.201972,71.200074,71.194917,71.177172,71.164140,71.167272,71.146998
2700~3000,71.065901,71.061526,71.070770,71.082445,71.062654,71.062939,71.075109,71.064312,71.064510,71.072112,...,71.233103,71.228460,71.222935,71.212641,71.220870,71.229811,71.242625,71.243267,71.255195,71.266497
3000~3300,71.017074,71.041419,71.025955,71.027903,71.037881,71.028722,71.027339,71.019331,71.035213,71.016046,...,71.175097,71.172520,71.184707,71.190117,71.182305,71.175160,71.178502,71.191878,71.188914,71.190434
3300~3600,71.055983,71.038720,71.036129,71.035033,71.038565,71.041735,71.043064,71.053810,71.037539,71.048879,...,71.242188,71.247572,71.243368,71.240028,71.247013,71.245267,71.229875,71.232344,71.234921,71.236625
3600~3900,71.050932,71.050931,71.057054,71.059760,71.056408,71.065414,71.060515,71.060518,71.057551,71.065115,...,71.182606,71.167307,71.162441,71.175044,71.175899,71.183058,71.190629,71.188636,71.176944,71.181275
3900~4200,71.063446,71.066416,71.053005,71.056334,71.057978,71.054954,71.044474,71.035722,71.056850,71.055918,...,71.121807,71.132597,71.121736,71.122565,71.113556,71.120765,71.124668,71.119775,71.128562,71.117719
4200~4500,71.062569,71.062557,71.068625,71.059284,71.065593,71.060912,71.074647,71.082941,71.066113,71.070773,...,71.202794,71.207855,71.216903,71.211276,71.213904,71.211043,71.215987,71.218693,71.206826,71.210765


환산점수_1.xlsx 생성 완료


'z_score_df : '

New_Measurement,1,2,3,4,5,6,7,8,9,10,...,270,271,272,273,274,275,276,277,278,279
TimeGroup,,,,,,,,,,,,,,,,,,,,,
1800~2100,54.787958,54.950022,54.280685,54.760498,54.897949,54.587100,54.223641,53.489595,53.832893,53.153193,...,72.248172,71.734752,71.758587,71.001030,70.926013,71.262803,71.058145,70.287438,69.825952,69.438733
2100~2400,52.597187,52.591457,52.563317,52.633032,52.591328,52.626706,52.578016,52.576295,52.557587,52.533635,...,66.730939,66.364288,64.724336,68.037963,67.755780,69.218454,68.745403,70.370887,70.448306,70.384485
2400~2700,52.518586,52.517123,52.505034,53.207126,52.513646,53.817562,53.469396,52.501334,52.587912,52.527196,...,94.009290,94.680817,95.038624,95.153870,94.234444,94.607440,94.573360,94.328615,94.363579,93.012966
2700~3000,54.076784,53.792124,53.509930,53.682174,53.455011,53.769199,53.910010,54.503649,53.118685,54.222431,...,93.090935,93.224181,92.665012,91.260327,91.191972,88.161591,86.359474,83.813393,82.121914,85.836900
3000~3300,53.194673,52.874585,52.964738,52.508215,53.480412,53.104116,53.333083,53.243530,53.473421,52.698827,...,88.580079,87.566793,86.544486,87.602944,89.716021,91.290669,91.573594,92.087265,91.366216,88.930622
3300~3600,53.206189,53.946587,53.121346,54.223122,52.946725,53.069714,53.160855,54.083629,54.174368,54.911642,...,70.848273,71.490349,80.013235,80.365804,72.113733,71.551674,71.575041,71.740637,71.707070,71.867574
3600~3900,52.627763,52.608476,52.590226,52.616502,52.581187,52.584306,52.597273,52.629515,52.622566,52.636922,...,70.020375,69.181297,67.465579,68.606280,67.373997,69.850527,70.214678,70.651403,70.356386,70.819010
3900~4200,55.086243,54.036549,55.289106,54.401347,54.344904,53.528109,53.414692,53.824547,53.045665,53.773422,...,86.480026,86.706766,88.598840,88.220705,88.976214,87.326222,85.941985,85.846750,89.151940,89.698176
4200~4500,54.654924,55.025003,54.405486,54.571869,54.347506,55.155138,54.628259,55.231666,56.965081,55.258358,...,71.862413,72.245565,71.855329,71.654084,71.982362,81.293571,82.910436,83.759798,72.397646,72.129107


...

New_Measurement,1,2,3,4,5,6,7,8,9,10,...,270,271,272,273,274,275,276,277,278,279
TimeGroup,,,,,,,,,,,,,,,,,,,,,
1800~2100,71.070465,71.057013,71.038639,70.997023,71.008176,70.989238,71.008338,70.979957,70.975927,70.978660,...,71.170585,71.150257,71.136262,71.122579,71.124066,71.130056,71.135538,71.152873,71.177423,71.173213
2100~2400,70.985406,70.978833,70.981882,71.020574,70.984146,70.977916,70.994015,71.038578,71.012130,70.993890,...,71.154797,71.166964,71.162922,71.168885,71.168417,71.165497,71.168667,71.171401,71.145001,71.145033
2400~2700,71.064393,71.047505,70.969057,70.975453,71.024106,71.014619,70.979079,70.957178,70.993654,70.982222,...,71.273065,71.273050,71.272444,71.264250,71.250599,71.239780,71.228926,71.218057,71.238762,71.243443
2700~3000,70.986469,70.963358,71.003251,70.995172,70.971790,70.972649,70.969390,70.987741,70.990732,70.980822,...,71.193774,71.185283,71.191280,71.188000,71.202618,71.201648,71.187429,71.182418,71.162498,71.173042
3000~3300,70.944056,70.935892,70.955537,70.952267,70.973241,70.947137,70.960606,70.974370,70.965821,70.977666,...,71.144615,71.147162,71.146910,71.152673,71.137254,71.138655,71.150580,71.158326,71.181112,71.194159
3300~3600,71.039918,71.005886,71.020531,71.016294,70.999752,71.003058,71.020078,71.002418,70.962802,70.988883,...,71.113099,71.119260,71.121783,71.115478,71.121827,71.121188,71.112375,71.113154,71.088936,71.074568
3600~3900,70.958241,70.936824,70.946952,70.963352,70.974601,70.934295,70.931045,70.919476,70.970597,70.940728,...,71.148966,71.159870,71.156101,71.176420,71.192113,71.208113,71.210082,71.209949,71.221128,71.213520
3900~4200,71.025982,71.066864,71.020382,70.968828,70.971927,71.015870,71.003234,71.022726,70.998620,70.996935,...,71.184409,71.179163,71.170477,71.156178,71.139619,71.123249,71.119005,71.107482,71.096570,71.101861
4200~4500,71.006861,70.951414,70.991104,70.994634,70.978752,70.984723,70.970124,70.972672,70.984882,71.006476,...,71.271497,71.266236,71.268792,71.271426,71.266022,71.263319,71.266026,71.276801,71.279466,71.278367


환산점수_4.xlsx 생성 완료


'z_score_df : '

New_Measurement,1,2,3,4,5,6,7,8,9,10,...,270,271,272,273,274,275,276,277,278,279
TimeGroup,,,,,,,,,,,,,,,,,,,,,
1800~2100,52.604175,52.764525,52.622278,52.672550,52.534361,52.558991,52.595609,52.542513,52.568917,52.568880,...,72.359120,72.471068,71.301573,71.876987,71.874796,81.639795,82.303971,81.592270,86.582491,86.222709
2100~2400,52.805752,52.600652,52.831957,52.751450,52.751462,52.751447,52.642973,52.885622,52.983469,52.818146,...,81.888444,82.707965,83.511487,84.318739,85.117458,82.273316,82.357102,81.922391,80.510397,71.156061
2400~2700,52.545623,52.532662,52.836151,52.635723,52.711480,52.607580,52.558668,52.545753,53.828137,53.839283,...,71.710974,71.234610,71.668043,71.662992,71.402581,71.030750,71.063200,71.218452,70.951365,72.174129
2700~3000,52.644380,52.658176,52.671865,52.586906,52.622896,52.671834,52.685500,52.599971,52.685547,52.699199,...,84.532844,88.590381,85.128284,84.734036,80.718093,72.343795,71.623916,72.315914,82.243510,80.607557
3000~3300,53.851600,54.691118,55.505219,54.661764,55.956615,54.489892,53.843056,54.011053,54.348737,54.347856,...,70.679348,70.451680,70.753015,70.485784,72.251858,80.457823,83.457750,80.150056,71.538611,82.334059
3300~3600,52.869439,52.814731,53.073197,52.853453,52.951672,52.797286,52.772221,52.801802,52.842770,52.842721,...,82.713910,83.515971,85.209180,85.490048,83.695174,83.282176,80.095050,84.814814,85.943288,81.970615
3600~3900,54.839884,52.669072,54.998225,52.833673,53.034337,53.402579,53.402456,52.603431,52.775748,52.568789,...,87.049829,85.829897,84.185602,81.081230,82.628441,85.788501,87.726044,85.101569,84.695693,85.207171
3900~4200,53.277676,52.734412,52.540223,52.807067,53.360459,52.554174,52.534623,54.737014,52.586153,53.207437,...,71.644752,70.717168,71.330858,71.332871,70.244178,70.336722,70.662369,70.660802,70.660729,70.396995
4200~4500,52.860101,52.763460,52.695678,52.724595,52.711771,52.751915,52.911100,52.791953,52.682945,52.711434,...,83.565045,87.815967,89.141016,91.019538,91.826112,90.551756,89.249363,86.300137,86.741493,89.756293


환산점수_5.xlsx 생성 완료
